# Qubit operations

In [2]:
using Revise
using Plots
using LinearAlgebra
using MultiScales

newaxis = [CartesianIndex()]

1-element Vector{CartesianIndex{0}}:
 CartesianIndex()

In [3]:
using ITensors

println(ITensors.blas_get_num_threads())

1


In [4]:
function eval(psi, sites, el)
    V = ITensor(1.)
    for j=1:length(el)
        V *= (psi[j] * state(sites[j], el[j]))
    end
    return scalar(V)
end

eval (generic function with 2 methods)

In [10]:
nbit = 1
N = 2^nbit

sites = siteinds("Qubit", nbit)

1-element Vector{Index{Int64}}:
 (dim=2|id=669|"Qubit,Site,n=1")

In [11]:
Z = [op("Z", sites, n) for n in 1:nbit]
;

In [12]:
mps = MPS(sites, ["1" for n in 1:nbit])

MPS
[1] ((dim=2|id=669|"Qubit,Site,n=1"),)


In [13]:
M = MPO(sites, ["Z" for n in 1:nbit])

MPO
[1] ((dim=2|id=669|"Qubit,Site,n=1")', (dim=2|id=669|"Qubit,Site,n=1"))


In [14]:
# Create Z_1 X_2
# See: https://github.com/ITensor/ITensors.jl/blob/main/src/mps/mpo.jl#L69
os = Prod{Op}()
os *= Op("Z", 1)
#os *= Op("X", 2)
M = MPO(os, sites)

MPO
[1] ((dim=2|id=669|"Qubit,Site,n=1")', (dim=2|id=669|"Qubit,Site,n=1"))


In [8]:
# Create Z_1
M = MPO(Op("Z", 1), sites)

MPO
[1] ((dim=3|id=380|"Link,l=1"), (dim=2|id=250|"Qubit,Site,n=1")', (dim=2|id=250|"Qubit,Site,n=1"))
[2] ((dim=3|id=380|"Link,l=1"), (dim=2|id=535|"Qubit,Site,n=2")', (dim=2|id=535|"Qubit,Site,n=2"))


In [16]:
@show M[1]

M[1] = ITensor ord=2
Dim 1: (dim=2|id=669|"Qubit,Site,n=1")'
Dim 2: (dim=2|id=669|"Qubit,Site,n=1")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2
 1.0   0.0
 0.0  -1.0


ITensor ord=2 (dim=2|id=669|"Qubit,Site,n=1")' (dim=2|id=669|"Qubit,Site,n=1")
NDTensors.Dense{Float64, Vector{Float64}}

In [ ]:
# Control: 1
# Target: 2
cnot = op("CNOT", sites, 1, 2)

state0 = MPS(sites, ["1" for n in 1:nbit])
for i2 in 1:2, i1 in 1:2
    @show i1, i2, eval(state0, sites, [i1, i2])
end
res = noprime(contract(MPO(cnot, sites), state0))

In [ ]:
for i2 in 1:2, i1 in 1:2
    @show i1, i2, eval(res, sites, [i1, i2])
end

In [ ]:
# Phase π: 1
# Phase π/2: 2
os = Prod{Op}()
os *= Op("Phase", 1; ϕ=π)
#os *= Op("Phase", 2; ϕ=π/2)
M = MPO(os, sites)
#M = MPO(Prod{Op}(Op("Phase", 1; ϕ=π)), sites)

state0 = MPS(sites, ["1" for n in 1:nbit])
for i2 in 1:2, i1 in 1:2
    @show i1, i2, eval(state0, sites, [i1, i2])
end
res = noprime(contract(M, state0))

#for i2 in 1:2, i1 in 1:2
    #@show i1, i2, eval(res, sites, [i1, i2])
#end

In [ ]:
replaceprime(contract(M, state0), 2=>1)

In [ ]:
apply(M, state0)

In [ ]:
state0 = MPS(sites, ["1" for n in 1:nbit])
for i2 in 1:2, i1 in 1:2
    @show i1, i2, eval(state0, sites, [i1, i2])
end
println("")
println("")

@show state0[1]
println("")
println("")

state0[1] = noprime(state0[1] * op("Phase", sites[1]; ϕ=π))
;

In [ ]:
for i2 in 1:2, i1 in 1:2
    @show "B ", i1, i2, eval(state0, sites, [i1, i2])
end

In [ ]:
sites = siteinds("Qubit", 4)
state0 = MPS(sites, ["1" for n in 1:4])
@show inds(state0[1])

In [ ]:
@show inds(state0[2])

In [ ]:
@show inds(state0[end])

In [ ]:
contract(M, state0)